# Exploratory Data Analysis

## Notebook Summary
In this notebook, I summarize where the data came from and some of its methodologies, and I look through some key columns in the dataset. I also explore important statistics for the project as a whole moving forward, and specifically examine distributions of various columns and how they relate to the target column.

## Data Source and Methodologies
The NSCH is a household survey that produces national and state-level data on the physical and emotional health of children 0 - 17 years old in the United States. Since 2016, the NSCH has been an annual survey. The survey supports national estimates every year and state-level estimates by combining 2 or 3 years of data. In this project I am looking at the 2020 data.

The survey collects information related to the health and well-being of children, including access to and use of health care, family interactions, parental health, school and after-school experiences, and neighborhood characteristics. A parent or other adult caregiver with knowledge of the sampled child’s health and health care filled out the topical questionnaire.

Survey topics include:
- Child and family characteristics
- **Physical and mental health status, including current conditions and functional difficulties**
- Health insurance status, type, and adequacy
- Access and use of health care services
- Medical, dental, and specialty care needed and received
- Family health and activities
- Impact of child’s health on family
- Neighborhood characteristics

Please see [this document](https://www2.census.gov/programs-surveys/nsch/technical-documentation/methodology/2020-NSCH-Methodology-Report.pdf) for a full report on the methodologies used by the US Census Bureau when obtaining this data.

## Loading and Exploring the Data
Lets first load and explore the data, and confirm its size and shape.

In [1]:
# Import statements
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Loading data into pandas dateframe
nsch = pd.read_sas('../Data/nsch_2020_topical_SAS/nsch_2020_topical.sas7bdat')

# Visually checking successful loading of dataframe
nsch.head()

/Users/matthewturner/opt/anaconda3/envs/Capstone/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")


,FIPSST,STRATUM,HHID,FORMTYPE,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,K2Q35A_1_YEARS,...,BIRTH_YR_F,BMICLASS,HHCOUNT_IF,FPL_I1,FPL_I2,FPL_I3,FPL_I4,FPL_I5,FPL_I6,FWC
0,b'17',b'1',b'20000003',b'T1',2.0,1.0,1.0,3.0,1.0,NaN,...,0.0,NaN,0.0,400.0,400.0,400.0,400.0,400.0,400.0,3296.080092
1,b'29',b'2A',b'20000004',b'T3',1.0,1.0,1.0,14.0,2.0,NaN,...,0.0,2.0,0.0,400.0,400.0,400.0,400.0,400.0,400.0,2888.545330
2,b'47',b'1',b'20000005',b'T1',1.0,1.0,1.0,1.0,2.0,NaN,...,0.0,NaN,0.0,400.0,400.0,400.0,400.0,400.0,400.0,1016.682730
3,b'28',b'1',b'20000014',b'T3',2.0,1.0,1.0,15.0,2.0,NaN,...,0.0,2.0,0.0,143.0,143.0,143.0,143.0,143.0,143.0,1042.091065
4,b'55',b'1',b'20000015',b'T3',2.0,2.0,1.0,16.0,2.0,NaN,...,0.0,3.0,0.0,400.0,400.0,400.0,400.0,400.0,400.0,402.372392


### Shape

In [13]:
# Getting shape of data frame and assigning to variables
total_rows, total_columns = nsch.shape

print(f'There are {total_rows} completed surveys')
print(f'There are {total_columns} questions in the survey')

There are 42777 completed surveys
There are 443 questions in the survey


The [Methodology Report](https://www2.census.gov/programs-surveys/nsch/technical-documentation/methodology/2020-NSCH-Methodology-Report.pdf) for this survey states that they ***reached out to 240,000 house holds, and 93,000 completed a screener survey**.

They also state that **of those 93,000 households, 51,107 are included in the public use file**, however, of the 51,107 households **only 42,777 submitted a complete survey**. So our numbers match up with what is in the report.

Lets get quick overview of the data before moving on

### Data Types

In [26]:
# Data Types in set
nsch.dtypes.value_counts()

float64    439
object       4
dtype: int64

Though most of these are floats, it is important to remember that this entire dataset is categorical data due to it being a survey.

In [29]:
nsch.isna().sum()

FIPSST       0
STRATUM      0
HHID         0
FORMTYPE     0
TOTKIDS_R    0
            ..
FPL_I3       0
FPL_I4       0
FPL_I5       0
FPL_I6       0
FWC          0
Length: 443, dtype: int64


## ADHD

Our target column for prediction will be "K2Q31A", which says:


> "Has a doctor or other health care provider EVER told you that this child has Attention Deficit Disorder or Attention-Deficit/Hyperactivity Disorder, that is, ADD or ADHD?"